# 大语言模型Transformer库-Pipeline组件实践

In [1]:
import os

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
os.environ['HF_HOME'] = '/root/autodl-tmp/cache/'

## 查询transformers库支持的所有任务类型

In [2]:
from transformers.pipelines import SUPPORTED_TASKS

SUPPORTED_TASKS.items()

/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dict_items([('audio-classification', {'impl': <class 'transformers.pipelines.audio_classification.AudioClassificationPipeline'>, 'tf': (), 'pt': (<class 'transformers.models.auto.modeling_auto.AutoModelForAudioClassification'>,), 'default': {'model': {'pt': ('superb/wav2vec2-base-superb-ks', '372e048')}}, 'type': 'audio'}), ('automatic-speech-recognition', {'impl': <class 'transformers.pipelines.automatic_speech_recognition.AutomaticSpeechRecognitionPipeline'>, 'tf': (), 'pt': (<class 'transformers.models.auto.modeling_auto.AutoModelForCTC'>, <class 'transformers.models.auto.modeling_auto.AutoModelForSpeechSeq2Seq'>), 'default': {'model': {'pt': ('facebook/wav2vec2-base-960h', '55bb623')}}, 'type': 'multimodal'}), ('text-to-audio', {'impl': <class 'transformers.pipelines.text_to_audio.TextToAudioPipeline'>, 'tf': (), 'pt': (<class 'transformers.models.auto.modeling_auto.AutoModelForTextToWaveform'>, <class 'transformers.models.auto.modeling_auto.AutoModelForTextToSpectrogram'>), 'defau

## Pipeline的使用

### 1、Pipeline的创建与使用方式

In [3]:
##导入pipeline
from transformers import pipeline
## 指定任务类型，再指定模型，创建基于指定模型的Pipeline
# https://huggingface.co/models
# roberta-base-finetuned-dianping-chinese：是专门为中文评论情感分析预训练和微调过的模型。
pipe = pipeline("text-classification", model="uer/roberta-base-finetuned-dianping-chinese")
pipe("我觉得不太行！")

[{'label': 'negative (stars 1, 2 and 3)', 'score': 0.9735506772994995}]

### 2、预先加载模型，再创建Pipeline

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 这种方式，必须同时指定model和tokenizer
model = AutoModelForSequenceClassification.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")
tokenizer = AutoTokenizer.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)
# 测试
pipe("今天天气很晴朗，处处好风光！")

[{'label': 'positive (stars 4 and 5)', 'score': 0.9193938374519348}]

### 3、CPU和GPU推理对比

#### 1）使用CPU推理

In [ ]:
pipe.model.device
#输出，默认使用CPU  device(type='cpu')

In [5]:
import torch
import time
times = []
for i in range(100):
    torch.cuda.synchronize()
    start = time.time()
    pipe("今天天气很晴朗，处处好风光！")
    torch.cuda.synchronize()
    end = time.time()
    times.append(end - start)
print(sum(times) / 100)

0.021346201896667482


#### 2）使用GPU进行推理

In [6]:
pipe = pipeline("text-classification", model="uer/roberta-base-finetuned-dianping-chinese", device=0)
pipe.model.device

device(type='cuda', index=0)

In [ ]:
import torch
import time
times = []
for i in range(100):
    torch.cuda.synchronize()
    start = time.time()
    pipe("我觉得不太行！")
    torch.cuda.synchronize()
    end = time.time()
    times.append(end - start)
print(sum(times) / 100)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


0.01271634578704834


### 4、确定Pipeline参数的使用

In [8]:
#创建了一个专门用于问答任务的Pipeline对象qa_pipe。问答任务是指系统根据给定的问题和上下文，找出并返回答案的任务。
#"question-answering"：指定了要执行的任务类型为问答。
#model="uer/roberta-base-chinese-extractive-qa"：指定了使用的模型，这个模型是专门为中文的抽取式问答（extractive QA）任务预训练过的模型。
qa_pipe = pipeline("question-answering", model="uer/roberta-base-chinese-extractive-qa")
qa_pipe

In [9]:
# question="云南的省会是哪里？"：提供了要回答的问题。
# context="云南的省会是昆明"：提供了回答问题所需的上下文信息。
# max_answer_len=2：指定了答案的最大长度。在这个例子中，设置为2意味着模型将尝试返回长度为2的最短可能答案。
qa_pipe(question="云南的省会是哪里？", context="云南的省会是昆明", max_answer_len=2)

{'score': 0.876051127910614, 'start': 6, 'end': 8, 'answer': '昆明'}

##### 结果参数说明：
- ‘score’：模型对找到的答案的置信度分数。在这个例子中，分数比较高，表明模型对这个答案是很有信心。
- ‘start’：答案在给定上下文中的起始索引。
- ‘end’：答案在给定上下文中的结束索引。
- ‘answer’：模型找到的答案，这里是根据索引从上下文中提取的文本片段。

## Pipeline背后的工作

### 1、导入依赖，创建模型

In [10]:
#这行代码尝试从transformers库中导入所有公共对象。通常，这种导入方式不推荐使用，因为它可能会导入大量不相关的对象，造成命名空间的混乱。
from transformers import *
import torch # PyTorch是一个开源的机器学习库，广泛用于深度学习。

# 加载与预训练模型相对应的分词器
tokenizer = AutoTokenizer.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")
# 加载预训练的文本分类模型。
model = AutoModelForSequenceClassification.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")

/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
loading configuration file config.json from cache at /root/autodl-tmp/cache/hub/models--uer--roberta-base-finetuned-dianping-chinese/snapshots/25faf1874b21e76db31ea9c396ccf2a0322e0071/config.json
Model config BertConfig {
  "_name_or_path": "uer/roberta-base-finetuned-dianping-chinese",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative (stars 1, 2 and 3)",
    "1": "positive (stars 4 and 5)"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative (stars 1, 2 and 3)": 0,
    "po

### 2、分词处理、生成模型输入

In [12]:
#定义了要进行分类的输入文本
input_text = "今天天气好晴朗，处处好风光！"
#使用分词器将输入文本转换为模型所需的格式，并指定返回的张量类型为PyTorch张量（"pt"代表PyTorch）
#这里的inputs变量将包含分词器处理后的结果，它是一个字典，包含分词器生成的编码张量、注意力掩码（attention mask）等信息
inputs = tokenizer(input_text, return_tensors="pt")
# 打印输出inputs变量，
inputs

{'input_ids': tensor([[ 101,  791, 1921, 1921, 3698, 1962, 3252, 3306, 8024, 1905, 1905, 1962,
         7599, 1045, 8013,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

### 3、模型推理

In [13]:
res = model(**inputs)
#打印结果
res

SequenceClassifierOutput(loss=None, logits=tensor([[-1.0786,  0.9897]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

### 4、获取原始分数、再归一化处理

In [15]:
##获取了模型的输出logits。Logits是模型在每个可能的类别上的得分或概率，还没有经过概率归一化处理。
logits = res.logits
## 使用PyTorch中的torch.softmax函数对logits进行softmax操作。softmax函数将每个logit值转换为在[0, 1]范围内的概率值，使得所有类别的概率之和为1
## dim=-1表示在最后一个维度上进行softmax操作。在情感分类任务中，通常最后一个维度是类别维度
logits = torch.softmax(logits, dim=-1)
logits

tensor([[0.1122, 0.8878]], grad_fn=<SoftmaxBackward0>)

### 5、查看概率最高的类别索引

In [18]:
#使用argmax找到logits中概率最高的类别索引，并通过.item()将其转换为一个普通的Python整数。
pred = torch.argmax(logits,-1).item()
#打印结果
pred

1

### 6、查看标签映射

In [19]:
#访问模型配置中的id2label映射，这是一个从索引到标签的映射
model.config.id2label

{0: 'negative (stars 1, 2 and 3)', 1: 'positive (stars 4 and 5)'}

### 7、根据我们得到的索引类别，从标签映射中取出结果

In [20]:
#使用预测得到的类别索引pred从id2label映射中获取对应的标签。
result = model.config.id2label.get(pred)
#打印标签结果
result

'positive (stars 4 and 5)'

### 8、Pipeline使用完整代码样例

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 加载预训练的Tokenizer和Model
tokenizer = AutoTokenizer.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")
model = AutoModelForSequenceClassification.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")

# 对输入文本进行编码
encoded_input = tokenizer("今天天气好晴朗，处处好风光！", return_tensors="pt")

# 模型推理
res = model(**encoded_input)

# 获取并处理logits
logits = res.logits
probabilities = torch.nn.functional.softmax(logits, dim=-1)

# 获取最可能的类别
pred = torch.argmax(probabilities).item()
#使用预测得到的类别索引pred从id2label映射中获取对应的标签。
result = model.config.id2label.get(pred)
#打印标签结果
result

## 使用Pipeline简化模型推理的代码

In [21]:
from transformers import pipeline

# 创建Pipeline，指定任务类型、模型和分词器
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

# 执行预测
optimized_result = pipe("今天天气好晴朗，处处好风光！")
print(optimized_result)

[{'label': 'positive (stars 4 and 5)', 'score': 0.8877794146537781}]
